In [47]:
# -*- coding: utf-8 -*-
"""
Created on Fri Nov 29 00:23:32 2024

@author: kam224
"""
import numpy as np
import matplotlib.pyplot as plt
from constants import *
from utils import *
from finiteElement.SpatialInterpolator import SpatialInterpolator
from finiteElement.Space import sparseAdvectionDiffSpace
import pickle
import argparse


plot = False
filepath = DEFAULT_FILEPATH

scale = "12_5k.txt"


nodes = np.loadtxt("../" + filepath + 'nodes_' + scale)
IEN = np.loadtxt("../" + filepath + 'IEN_' + scale, dtype=np.int64)
#correction for IEN ordering
IEN = IEN[:,-1::-1]
boundaries = np.loadtxt("../" + filepath + 'bdry_' + scale, 
                            dtype=np.int64)

with open("model.pkl", "rb") as f:
    interp = pickle.load(f)
print("Loaded Successfully")
df = interp.interpolate(nodes.T,[0])[["horizontal_wind_speed", "vertical_wind_speed"]]
initialVelocityMatrix = df[["horizontal_wind_speed", "vertical_wind_speed"]].to_numpy()

#initialize the weights for the weights mask with which Psi at reading is estimated with
weights = gaussianWeights(nodes, READING_LOC)
weights = weights/np.sum(weights)

#initialize the space
space = sparseAdvectionDiffSpace(nodes, IEN, boundaries, initialVelocityMatrix, 10e-5)
space.assemble(gaussianSource)

timestepsPerHour = int(3600/1)
 
space.timestep(0.1)
psi = space.getPsi()



Loaded Successfully


c:\Users\kam224\OneDrive - Imperial College London\Documents\NumericsProject2\venv\Lib\site-packages\scipy\sparse\_index.py:151: SparseEfficiencyWarning: Changing the sparsity structure of a csc_array is expensive. lil and dok are more efficient.
  self._set_arrayXarray(i, j, x)


In [59]:
for i in range(int(timestepsPerHour/10) - 1):
        space.timestep(10)
if space.cur_t > 8 * 3600:
    space.forceVector = np.zeros(space.forceVector.shape)
df = interp.interpolate(nodes.T,[space.cur_t/3600.])[["horizontal_wind_speed", "vertical_wind_speed"]]
updatedVelocityMatrix = df.to_numpy()
space.timestep(1, updatedVelocityMatrix)
print(space.cur_t)
psi = space.getPsi()

25146.1


In [36]:
23450.1/3600

6.513916666666666

In [37]:
weights[weights < 1e-5] = 0

In [64]:
np.ones((5, 20)).mean(axis = 0)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1.])

In [38]:
np.sum(psi)

np.float64(142.70947085991247)